In [1]:
import numpy as np
import math
from qiskit import QuantumCircuit, QuantumRegister, execute, Aer
from qiskit.providers.aer import QasmSimulator
import qiskit_create_circ as qcc

In [ ]:
# dir(qcc)

In [2]:
N = 1
m = 0
n_i = 1
L = int(math.floor(math.log(N + n_i, 2)) + 1)
print(L)
# l = int(math.ceil(math.log(m + n_i, 2)) )
l = int(math.floor(math.log(m + n_i, 2)) + 1)
print(l)



pReg, hReg, w_hReg, eReg, wReg, n_aReg, w_aReg, n_bReg, w_bReg, n_phiReg, w_phiReg = qcc.allocateQubits(N, n_i, L)

# circuit = QuantumCircuit(pReg, hReg, w_hReg, eReg, wReg, n_aReg, w_aReg, n_bReg, w_bReg, n_phiReg, w_phiReg)
circuit = QuantumCircuit(pReg, hReg, w_hReg, eReg, wReg, n_aReg, w_aReg, n_bReg, w_bReg, n_phiReg, w_phiReg)

simulator = Aer.get_backend('statevector_simulator')

2
1
nqubits_a_b_phi:  2


In [ ]:
print(w_phiReg)

In [3]:
g_1, g_2, g_12 = 2, 1, 0
gp = math.sqrt(abs((g_1 - g_2) ** 2 + 4 * g_12 ** 2))
if g_1 > g_2:
    gp = -gp
g_a, g_b = (g_1 + g_2 - gp) / 2, (g_1 + g_2 + gp) / 2
u = math.sqrt(abs((gp + g_1 - g_2)/ (2 * gp)))
eps = .001

timeStepList, P_aList, P_bList, P_phiList, Delta_aList, Delta_bList, Delta_phiList = [], [], [], [], [], [], []
qcc.populateParameterLists(N, timeStepList, P_aList, P_bList, P_phiList, Delta_aList, Delta_bList, 
                       Delta_phiList, g_a, g_b, eps)


Test uCount

In [ ]:
# qcc.uCount(circuit, m, n_i, l, pReg, wReg, n_aReg, w_aReg, n_bReg, w_bReg, n_phiReg, w_phiReg)

# result = execute(circuit, simulator).result()
# statevector = result.get_statevector(circuit)

# print(statevector)

Test uE

In [4]:
qcc.uE(circuit, l, n_i, m, n_phiReg, w_phiReg, n_aReg, w_aReg, n_bReg, w_bReg, wReg, eReg,
           Delta_phiList[0], Delta_aList[0], Delta_bList[0])

result = execute(circuit, simulator).result()
statevector = result.get_statevector(circuit)

print(statevector)
print(len(statevector))

counts list:  [[0, 0, 1], [0, 1, 0], [1, 0, 0]]
length counts list:  3


IndexError: list index out of range

In [ ]:
print(statevector**2)

In [ ]:
circuit.draw()

In [ ]:
print(l)

In [ ]:
circ = QuantumCircuit(pReg, hReg, w_hReg, eReg, wReg, n_aReg, w_aReg, n_bReg, w_bReg, n_phiReg, w_phiReg)

In [ ]:
qcc.numberControl(circ, l, 0, n_phiReg, w_phiReg)

In [ ]:
circ.draw()

Test U_h

In [5]:
# print(P_bList)
qcc.U_h(circuit, l, n_i, m, n_phiReg, w_phiReg, n_aReg, w_aReg, n_bReg, w_bReg, wReg, eReg, pReg, hReg, w_hReg,
            P_phiList[0], P_aList[0], P_bList[0])

Qubit(QuantumRegister(6, 'q0'), 2)
Qubit(QuantumRegister(5, 'q4'), 1)
target index:  2
l:  1 
k:  1
Qubit(QuantumRegister(6, 'q0'), 2)
Qubit(QuantumRegister(5, 'q4'), 1)
target index:  2
l:  1 
k:  1
l:  1 
k:  1
Qubit(QuantumRegister(6, 'q0'), 2)
Qubit(QuantumRegister(5, 'q4'), 1)
target index:  2
l:  1 
k:  1
Qubit(QuantumRegister(6, 'q0'), 2)
Qubit(QuantumRegister(5, 'q4'), 1)
target index:  2
l:  1 
k:  1
l:  1 
k:  1
Qubit(QuantumRegister(6, 'q0'), 2)
Qubit(QuantumRegister(5, 'q4'), 1)
target index:  2
l:  1 
k:  1
Qubit(QuantumRegister(6, 'q0'), 2)
Qubit(QuantumRegister(5, 'q4'), 1)
target index:  2
l:  1 
k:  1
l:  1 
k:  1
Qubit(QuantumRegister(6, 'q0'), 2)
Qubit(QuantumRegister(5, 'q4'), 1)
target index:  0
Qubit(QuantumRegister(6, 'q0'), 2)
Qubit(QuantumRegister(5, 'q4'), 1)
target index:  0


In [6]:
result = execute(circuit, simulator).result()
statevector = result.get_statevector(circuit)

print(statevector)

[-4.13030347e-16-1.2246468e-15j  0.00000000e+00+0.0000000e+00j
  0.00000000e+00+0.0000000e+00j ...  0.00000000e+00+0.0000000e+00j
  0.00000000e+00+0.0000000e+00j  0.00000000e+00+0.0000000e+00j]


In [7]:
circuit.draw()

┌───┐                       »
 q0_0: ──────────────────────■──────────────────■──┤ X ├───────────────────────»
       ┌───┐                 │                  │  ├───┤┌───┐                  »
 q0_1: ┤ X ├─────────────────■──────────────────■──┤ X ├┤ X ├──────────────────»
       ├───┤                 │                  │  ├───┤└───┘                  »
 q0_2: ┤ X ├─────────────────┼─────────────■────┼──┤ X ├───────────────────────»
       └───┘                 │             │    │  └───┘                       »
 q0_3: ──────────────────────┼─────────────┼────┼──────────────────────────────»
                             │             │    │                              »
 q0_4: ──────────────────────┼─────────────┼────┼──────────────────────────────»
                             │             │    │                              »
 q0_5: ──────────────────────┼─────────────┼────┼──────────────────────────────»
                             │  ┌───────┐  │    │            ┌───────┐┌───────┐»
 q1_0: ──────────────────────┼──┤ RY(π) ├──┼────┼────────────┤ RY(π) ├┤ RY(π) ├»
                             │  └───┬───┘  │    │            └───┬───┘└───┬───┘»
 q2_0: ──────────────────────┼──────┼──────┼────┼────────────────┼────────┼────»
                             │      │      │    │                │        │    »
 q3_0: ──────────────────────┼──────┼──────┼────┼─────────■──────┼────────┼────»
            ┌───┐            │      │      │    │         │      │        │    »
 q4_0: ─────┤ X ├───────■────┼──────■──────┼────┼─────────┼──────┼────────┼────»
            └─┬─┘     ┌─┴─┐┌─┴─┐           │  ┌─┴─┐       │      │        │    »
 q4_1: ───────┼───────┤ X ├┤ X ├───────────■──┤ X ├──■────┼──────■────────┼────»
              │       └─┬─┘└───┘         ┌─┴─┐└───┘  │    │               │    »
 q4_2: ───────┼─────────┼────────────────┤ X ├───────■────┼───────────────■────»
              │         │                └───┘     ┌─┴─┐  │                    »
 q4_3: ───────┼─────────┼──────────────────────────┤ X ├──■────────────────────»
              │         │                          └───┘┌─┴─┐                  »
 q4_4: ───────┼─────────┼───────────────────────────────┤ X ├──────────────────»
       ┌───┐  │         │                               └───┘                  »
 q5_0: ┤ X ├──■─────────┼──────────────────────────────────────────────────────»
       ├───┤  │  ┌───┐  │                                                      »
 q5_1: ┤ X ├──┼──┤ X ├──┼──────────────────────────────────────────────────────»
       └───┘  │  └───┘  │                                                      »
 q6_0: ───────┼─────────┼──────────────────────────────────────────────────────»
              │         │                                                      »
 q6_1: ───────┼─────────┼──────────────────────────────────────────────────────»
              │         │                                                      »
 q7_0: ───────┼─────────■──────────────────────────────────────────────────────»
       ┌───┐  │  ┌───┐                                                         »
 q7_1: ┤ X ├──┼──┤ X ├─────────────────────────────────────────────────────────»
       └───┘  │  └───┘                                                         »
 q8_0: ───────┼────────────────────────────────────────────────────────────────»
              │                                                                »
 q8_1: ───────┼────────────────────────────────────────────────────────────────»
       ┌───┐  │                                                                »
 q9_0: ┤ X ├──■────────────────────────────────────────────────────────────────»
       ├───┤┌───┐                                                              »
 q9_1: ┤ X ├┤ X ├──────────────────────────────────────────────────────────────»
       └───┘└───┘                                                              »
q10_0: ────────────────────────────────────────────────────────────────────────»
 

Testing U_p

In [ ]:
qcc.U_p(circuit, l, n_i, 0, pReg, hReg, w_hReg, wReg, g_a, g_b)

# result = execute(circuit, simulator).result()
# statevector = result.get_statevector(circuit)

# print(statevector)